# Focused Coding
___

## Table of Content

1. [Libraries](#libraries)
2. [Data Preprocessing](#preprocessing-data-and-grouping)
3. [Focused Coding Helper](#focused-coding----sample-helper)
___

## Libraries

All libraries which are needed to execute the code are listed here. Install the packages by using the `requirements.txt` file. 

The documentation can be found in the [README.md](README.md) file.

In [3]:
# import packages
import pandas as pd 
import os
import numpy as np
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import nltk
from preprocessing_functions import *

## Preprocessing Data and Grouping

In [4]:
# load data
df = pd.read_csv('data/comments_final.csv')
df.head(3)

,video_id,published_at,like_count,text,author
0,uW6fi2tCnAc,2023-02-19T21:22:45Z,1,"The answer is if China and India don't help, i...",0.0
1,uW6fi2tCnAc,2023-02-19T00:43:40Z,2,"and that guy is an expert, we're screwed",1.0
2,uW6fi2tCnAc,2023-02-18T22:57:38Z,4,Kennedy is a gem.,2.0


In [5]:
# group data by author and see distribution of comments 
df['author'] = pd.to_numeric(df['author'], errors='coerce').astype('Int64')
summary = df.groupby('author').agg(
    count=('author', 'size'),
    unique_video_id_count=('video_id', 'nunique')
).reset_index()
summary.sort_values(by='count', ascending=False, inplace=True)

# print top 20 authors
latex_table = summary.head(20).to_latex(index=False)
print(latex_table)

\begin{tabular}{rrr}
\toprule
author & count & unique_video_id_count \\
\midrule
17605 & 206 & 206 \\
3743 & 188 & 124 \\
18119 & 166 & 161 \\
17604 & 155 & 138 \\
18380 & 154 & 154 \\
17977 & 142 & 138 \\
17906 & 139 & 115 \\
17676 & 135 & 103 \\
17755 & 134 & 134 \\
14017 & 129 & 126 \\
14057 & 128 & 123 \\
2610 & 125 & 125 \\
25732 & 116 & 88 \\
1645 & 107 & 62 \\
17581 & 106 & 104 \\
6165 & 103 & 52 \\
6 & 99 & 98 \\
1106 & 97 & 75 \\
17608 & 92 & 80 \\
17590 & 87 & 79 \\
\bottomrule
\end{tabular}



In [6]:
# extract original text for seeing in comparising capitalized words etc.
extracted_col = df["text"]

# process data with using functions from functions.py
processed_df = (
    df.pipe(remove_users, 'text')
      .pipe(lowercase_text, 'text')
      .pipe(remove_whitespace, 'text')
      .pipe(remove_punctuation, 'text')
)

In [7]:
# Add the extracted column to the second DataFrame
processed_df = pd.concat([processed_df, extracted_col.rename("og_text")], axis=1)
processed_df.head(3)

,video_id,published_at,like_count,text,author,og_text
0,uW6fi2tCnAc,2023-02-19T21:22:45Z,1,the answer is if china and india dont help it ...,0,"The answer is if China and India don't help, i..."
1,uW6fi2tCnAc,2023-02-19T00:43:40Z,2,and that guy is an expert were screwed,1,"and that guy is an expert, we're screwed"
2,uW6fi2tCnAc,2023-02-18T22:57:38Z,4,kennedy is a gem,2,Kennedy is a gem.


In [8]:
# use lemmatization to reduce words to their root form
processed_df['text'] = processed_df['text'].astype('str')
processed_df = lemmatize_words(processed_df, 'text')

In [9]:
processed_df.lemmatized_text = processed_df.lemmatized_text.apply(lambda x: '' if str(x) == 'nan' else x)

## Focused Coding -- Sample helper

this helper is written to find comments with important keywords we could firstly find through exploring the comments or through topic modeling and Word2Vec.

How does it work?

- Input: Insert a lemmatized keyword of interest or multiple in `substrings` or `string`
- Ouput: Get comments as output which are talking about those keywords to get a variety of comments based on that topic or word of interest.

In [14]:
substrings = ['god plan', 'greenhouse gas', 'natural cycle', 'hoax']
string = ['climate model']
pattern = '|'.join(string)

In [15]:
filtered_df = processed_df[processed_df['lemmatized_text'].str.contains(pattern, case=False, na=False)]
pd.set_option('display.max_colwidth', None)
print(len(filtered_df))
filtered_df.sample(n=10)

202


,video_id,published_at,like_count,text,author,og_text,lemmatized_text
15320,7LVSrTZDopM,2023-02-02T16:18:31Z,2,would have been interesting to hear what dr lindzen thinks about copernicus satellite data with the amount of different measurements and level of detail to calibrate any digital model the uncertainty margins that are included in current digital models and have been decreasing over the years based on these data the fact that currently there is more mass created by men than biological mass bacteria plants animals on earth if buffers to absorb max and min temperatures and humidity like in forests especially rainforest would exist research papers that say some species are close to extinction why the amount of flooding events and droughtfires seems so much higher over the last decade to me it is also clear that politics has oversimplified the climate model by just focusing on one of the many greenhouse gasses and ignoring buffers or other disturbing processes but there is certainly effect of what we western humans have been doing during the last century e i think rich people will survive by climate engineering but many animals plants and other parts of earths beauty will be destroyed not quite something to be proud of id say,12701,"Would have been interesting to hear what Dr. Lindzen thinks about:\n-Copernicus satellite data with the amount of different measurements and level of detail to calibrate any digital model\n-The uncertainty margins that are included in current digital models and have been decreasing over the years, based on these data\n-The fact that currently there is more mass created by men than biological mass (bacteria, plants, animals) on Earth\n-If buffers to absorb max. and min. temperatures and humidity, like in forests (especially Rainforest) would exist\n-Research papers that say some species are close to extinction\n-Why the amount of flooding events and drought/fires seems so much higher over the last decade\n\nTo me it is also clear that politics has oversimplified the climate model by just focusing on one of the many greenhouse gasses and ignoring buffers or other disturbing processes, but there is certainly effect of what we, (Western) humans, have been doing during the last century. E \nI think rich people will survive by climate engineering, but many animals, plants and other parts of Earth's beauty will be destroyed. Not quite something to be proud of, I'd say.",would have be interest to hear what dr lindzen think about copernicus satellite data with the amount of different measurement and level of detail to calibrate any digital model the uncertainty margin that be include in current digital model and have be decrease over the year base on these data the fact that currently there be more mass create by men than biological mass bacteria plant animal on earth if buffer to absorb max and min temperature and humidity like in forest especially rainforest would exist research paper that say some specie be close to extinction why the amount of flood event and droughtfires seem so much high over the last decade to me it be also clear that politics have oversimplify the climate model by just focus on one of the many greenhouse gas and ignore buffer or other disturbing process but there be certainly effect of what we western human have be do during the last century e i think rich people will survive by climate engineering but many animal plant and other part of earths beauty will be destroy not quite something to be proud of id say
25960,niZdc2BEuqY,2024-05-11T00:12:35Z,0,all the climate models are bogus none can replicate the historic climate record because they all think co2 drives the climate,18017,All the climate models are bogus. None can replicate the historic climate record (because they all think CO2 drives the climate!),all the climate model be bogus none can replicate the historic climate record because they all think co2 drive the climate
62574,U0Xqu6BJHkk,2023-11-15T17:22:43Z,2,being agnosti